In [1]:
from prepare_data_for_modeling_or import ModelingData
import statsmodels.api as sm
from constants import *
import numpy as np
from sklearn.metrics import accuracy_score


In [2]:
obj = ModelingData()

File already exists: reading now...
File already exists: reading now...
File already exists: reading now...


In [3]:
mdf = obj.get_data_with_pop()

In [4]:
mdf['POVERTY_GUIDE'].sum()

13333118090.0

In [5]:
def fit_logistic(X, y):
    res = sm.Logit(y, X).fit()
    params = res.params
    conf = res.conf_int()
    conf['Odds Ratio'] = params
    conf.columns = ['5%', '95%', 'Odds Ratio']
    prediction = res.predict(X).apply(lambda x: 0 if x<0.5 else 1).tolist()
    print("Accuracy: ", accuracy_score(y,prediction))
    print(np.exp(conf))
    return conf, res

In [6]:
sc = MODELING_COLUMNS + ['_INCOMG']

In [7]:
df = mdf[sc]

In [8]:
df

,TAILPIPE,_AGEG5YR,SEX,_RACE_G1,POVERTY,POPEST2017_CIV,_EDUCAG,_BMI5CAT,ASTHMA,_INCOMG
0,0,5.0,1.0,1.0,0.0,-0.204798,2.0,2.0,0,3.0
1,0,8.0,2.0,1.0,0.0,-0.204798,2.0,3.0,0,5.0
2,0,1.0,1.0,1.0,NaN,-0.204798,2.0,2.0,0,9.0
3,0,10.0,2.0,1.0,NaN,-0.204798,2.0,NaN,0,9.0
4,0,12.0,2.0,1.0,0.0,-0.204798,3.0,4.0,1,5.0
...,...,...,...,...,...,...,...,...,...,...
747245,0,7.0,1.0,3.0,1.0,NaN,2.0,3.0,0,2.0
747246,0,2.0,2.0,3.0,0.0,NaN,4.0,4.0,1,3.0
747247,0,11.0,2.0,3.0,NaN,NaN,3.0,1.0,0,9.0
747248,0,10.0,2.0,3.0,0.0,NaN,4.0,NaN,0,3.0


In [9]:
# keeping the same method for fillna
for column in df.columns:
    if df[column].isna().mean() > 0:
        print(column)
        df[column] = df[column].fillna(df[column].mode()[0])

_RACE_G1
POVERTY
POPEST2017_CIV
_BMI5CAT


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
# A: just adding poverty/no income
X = df.drop(['ASTHMA'], axis=1)
y = df['ASTHMA']
conf, res = fit_logistic(X, y)

Optimization terminated successfully.
         Current function value: 0.297962
         Iterations 7
Accuracy:  0.9141237872198059
                      5%       95%  Odds Ratio
TAILPIPE        1.080235  1.120927    1.100393
_AGEG5YR        0.932296  0.936237    0.934264
SEX             1.048688  1.077385    1.062940
_RACE_G1        0.805283  0.819884    0.812551
POVERTY         0.796461  0.845339    0.820536
POPEST2017_CIV  0.949196  0.963132    0.956139
_EDUCAG         0.741512  0.752329    0.746901
_BMI5CAT        0.941968  0.957349    0.949627
_INCOMG         0.877797  0.884642    0.881213


In [17]:
# : just adding poverty removing income
X = df.drop(['ASTHMA', '_INCOMG', 'POPEST2017_CIV'], axis=1)
y = df['ASTHMA']
conf, res = fit_logistic(X, y)

Optimization terminated successfully.
         Current function value: 0.300981
         Iterations 7
Accuracy:  0.9141237872198059
                5%       95%  Odds Ratio
TAILPIPE  1.021190  1.058635    1.039744
_AGEG5YR  0.928974  0.932861    0.930915
SEX       0.961610  0.987870    0.974651
_RACE_G1  0.770755  0.784736    0.777714
POVERTY   1.169568  1.236341    1.202491
_EDUCAG   0.691458  0.700922    0.696174
_BMI5CAT  0.893516  0.907794    0.900627


In [12]:
df2 = mdf[sc]

In [13]:
df2.isna().mean()

TAILPIPE          0.000000
_AGEG5YR          0.000000
SEX               0.000000
_RACE_G1          0.017295
POVERTY           0.166773
POPEST2017_CIV    0.019281
_EDUCAG           0.000000
_BMI5CAT          0.074042
ASTHMA            0.000000
_INCOMG           0.000000
dtype: float64

In [14]:
df2 = df2.dropna()

In [15]:
# C: [dropping na]just adding poverty
X = df2.drop(['ASTHMA'], axis=1)
y = df2['ASTHMA']
conf, res = fit_logistic(X, y)

Optimization terminated successfully.
         Current function value: 0.294294
         Iterations 7
Accuracy:  0.9143583609630531
                      5%       95%  Odds Ratio
TAILPIPE        1.123345  1.172146    1.147486
_AGEG5YR        0.923133  0.927688    0.925407
SEX             1.068258  1.102303    1.085147
_RACE_G1        0.812318  0.829328    0.820779
POVERTY         0.615151  0.657701    0.636070
POPEST2017_CIV  0.943915  0.959894    0.951871
_EDUCAG         0.841675  0.858103    0.849849
_BMI5CAT        1.008834  1.027701    1.018224
_INCOMG         0.732551  0.743244    0.737878


In [16]:
# D: just adding poverty removing income
X = df2.drop(['ASTHMA', '_INCOMG'], axis=1)
y = df2['ASTHMA']
conf, res = fit_logistic(X, y)

Optimization terminated successfully.
         Current function value: 0.300082
         Iterations 7
Accuracy:  0.9143583609630531
                      5%       95%  Odds Ratio
TAILPIPE        1.058027  1.103684    1.080614
_AGEG5YR        0.924192  0.928775    0.926481
SEX             0.993199  1.024748    1.008850
_RACE_G1        0.785600  0.802369    0.793940
POVERTY         1.142097  1.214616    1.177798
POPEST2017_CIV  0.950729  0.966666    0.958664
_EDUCAG         0.667665  0.678288    0.672956
_BMI5CAT        0.909310  0.925598    0.917418
